In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_path = "/content/drive/My Drive/emotion_detection_model"
emotion_tokenizer = AutoTokenizer.from_pretrained(model_path)
emotion_model = AutoModelForSequenceClassification.from_pretrained(model_path)

emotion_model.eval()


id_to_label = emotion_model.config.id2label

def predict_emotion(text: str) -> str:
    """
    Predicts the emotion of the given text using a pre-trained emotion classification model.

    Args:
        text (str): The input text to classify.

    Returns:
        str: The predicted emotion label.
    """
    
    inputs = emotion_tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = {key: value.to(device) for key, value in inputs.items()}
    emotion_model.to(device)

    with torch.no_grad():
        outputs = emotion_model(**inputs)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=1).item()

    return id_to_label[predicted_class_id]

sample_texts = [
    "I can't believe I got the promotion! This is a dream come true after all the hard work.",
    "The loss of my pet has left an emptiness I can't explain. It's been a tough few weeks.",
    "I'm beyond furious with the way customer service handled my issue—it was utterly unacceptable!",
    "Honestly, I feel so uncertain about the future, like I’m just drifting without purpose.",
    "The sunset was so beautiful today—it reminded me how grateful I am for small moments of peace.",
    "It's strange. I feel both excited and scared about starting a new chapter in my life.",
    "I’ve been feeling neglected lately, like no one really notices or cares about me anymore.",
    "What an amazing surprise! I didn’t expect my friends to throw such a thoughtful party.",
    "It's been a monotonous day, nothing exciting, just going through the motions as usual.",
    "I can’t describe how anxious I feel about the upcoming interview—it’s overwhelming."
]

for text in sample_texts:
    print(f"Text: {text}")
    print(f"Predicted Emotion: {predict_emotion(text)}")
    print()


Text: I can't believe I got the promotion! This is a dream come true after all the hard work.
Predicted Emotion: surprise

Text: The loss of my pet has left an emptiness I can't explain. It's been a tough few weeks.
Predicted Emotion: sadness

Text: I'm beyond furious with the way customer service handled my issue—it was utterly unacceptable!
Predicted Emotion: anger

Text: Honestly, I feel so uncertain about the future, like I’m just drifting without purpose.
Predicted Emotion: confusion

Text: The sunset was so beautiful today—it reminded me how grateful I am for small moments of peace.
Predicted Emotion: gratitude

Text: It's strange. I feel both excited and scared about starting a new chapter in my life.
Predicted Emotion: fear

Text: I’ve been feeling neglected lately, like no one really notices or cares about me anymore.
Predicted Emotion: disappointment

Text: What an amazing surprise! I didn’t expect my friends to throw such a thoughtful party.
Predicted Emotion: admiration

Te

In [3]:
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

Found existing installation: unsloth 2024.12.4
Uninstalling unsloth-2024.12.4:
  Successfully uninstalled unsloth-2024.12.4
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-i1qd66hu
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-i1qd66hu
  Resolved https://github.com/unslothai/unsloth.git to commit 85f1fa096afde5efe2fb8521d8ceec8d13a00715
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.12.4-py3-none-any.whl size=173746 sha256=bba443cc3399416557f64406946c03563af0e69576feb8bc98bc66b85dac6a91
  Stored in directory: /tmp/pip-ephem-wheel-cache-n43g_byk/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [4]:
!pip install -U bitsandbytes

In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 
dtype = None 
load_in_4bit = True 


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [6]:
!pip install torch torchvision torchaudio


In [7]:
# Import necessary libraries
import torch
from unsloth import FastLanguageModel

# -------------------------------
# Mental Health Exercise Generation Setup
# -------------------------------

# Configuration Parameters for LLaMA Model
max_seq_length = 2048  # Maximum sequence length
temperature = 0.7      # Sampling temperature
top_p = 0.9            # Nucleus sampling parameter
top_k = 50             # Top-K sampling parameter
max_new_tokens = 150   # Maximum number of tokens to generate

# Define the Alpaca-style prompt template with a placeholder for the emotion
alpaca_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Provide a mental health exercise for the following feeling.

### Input:
{input_text}

### Response:
"""

# Enable faster inference (if supported by the library)
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

# **Do not manually move the LLaMA model to the device**; let unsloth handle it
# Remove or comment out the following lines to prevent conflicts:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# llama_model.to(device)
# llama_model.eval()

def generate_alpaca_prompt(emotion: str) -> str:
    """
    Generates the Alpaca-style prompt by inserting the emotion into the template.

    Args:
        emotion (str): The emotion label based on which the exercise will be generated.

    Returns:
        str: The formatted prompt.
    """
    return alpaca_prompt_template.format(input_text=emotion)

def generate_exercise(emotion: str,
                     max_new_tokens: int = max_new_tokens,
                     temperature: float = temperature,
                     top_p: float = top_p,
                     top_k: int = top_k) -> str:
    """
    Generates a mental health exercise based on the provided emotion using the LLaMA model.

    Args:
        emotion (str): The emotion label based on which the exercise will be generated.
        max_new_tokens (int): Maximum number of tokens to generate.
        temperature (float): Sampling temperature.
        top_p (float): Nucleus sampling parameter.
        top_k (int): Top-K sampling parameter.

    Returns:
        str: The generated mental health exercise.
    """
    try:
        # Construct the prompt by filling in the emotion
        prompt = generate_alpaca_prompt(emotion)

        # **Optional Debugging: Print the formatted prompt to verify correctness**
        # Uncomment the next line to see the prompt
        # print(f"Generated Prompt:\n{prompt}\n")

        # Tokenize the prompt using the LLaMA tokenizer
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=max_seq_length
        )

        # Generate the response
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                top_k=top_k,
                do_sample=True,  # Enable sampling for creativity
                eos_token_id=tokenizer.eos_token_id,
            )

        # Decode the generated tokens
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # **Optional Debugging: Print the generated text to verify correctness**
        # Uncomment the next line to see the generated text
        # print(f"Generated Text:\n{generated_text}\n")

        # Extract the response portion by removing the prompt
        response = generated_text[len(prompt):].strip()

        # **Optional: Further clean the response if necessary**
        # For example, remove any trailing punctuation
        response = response.rstrip('.') if response.endswith('.') else response

        return response

    except Exception as e:
        print(f"Error during exercise generation: {e}")
        return "Unable to generate exercise at this time."

# -------------------------------
# Integrated Emotion Classification and Exercise Generation
# -------------------------------

# Example Usage
if __name__ == "__main__":
    # Define your input texts for testing
    sample_texts = [
        "I just got promoted at work! I'm so excited and happy.",
        "I'm feeling really down today. Nothing seems to go right.",
        "That movie was absolutely terrifying. I couldn't sleep afterwards.",
        "I'm so grateful for all the support from my friends and family.",
        "The news about the storm has left me anxious and worried."
    ]

    # Iterate over each sample text
    for idx, text in enumerate(sample_texts, 1):
        print(f"--- Example {idx} ---")
        print(f"Input Text: {text}")

        # Step 1: Predict Emotion (Assumed to be defined and run in a previous cell)
        emotion = predict_emotion(text)
        print(f"Predicted Emotion: {emotion}")

        # Step 2: Generate Mental Health Exercise based on Emotion
        exercise = generate_exercise(emotion)
        print(f"Mental Health Exercise: {exercise}\n")

    # Optional: Interactive Loop for Dynamic Inputs
    while True:
        print("\n--- New Input ---")
        user_input = input("Enter the text to classify its emotion (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            print("Exiting the emotion classification and exercise generation loop.")
            break

        # Step 1: Predict Emotion
        user_emotion = predict_emotion(user_input)
        print(f"\nPredicted Emotion: {user_emotion}")

        # Step 2: Generate Mental Health Exercise based on Emotion
        user_exercise = generate_exercise(user_emotion)
        print(f"Mental Health Exercise: {user_exercise}\n")


--- Example 1 ---
Input Text: I just got promoted at work! I'm so excited and happy.
Predicted Emotion: excitement
Mental Health Exercise: Exercise:
1. Think about a time you were excited about something
2. What was it?
3. What was your body doing?
4. Did your excitement motivate you to act?
5. What were you doing at the time?
6. Did your excitement motivate you to do something?
7. How did you feel?
8. Did you have any physical symptoms?
9. Did you have any emotional symptoms?
10. Did your excitement motivate you to take action?
11. Did you have any thoughts?
12. Did your excitement motivate you to take action?
13. Did you have any physical symptoms?
14. Did you have any emotional symptoms?
15. Did your excitement motivate you to take action?
16. Did you have any thoughts

--- Example 2 ---
Input Text: I'm feeling really down today. Nothing seems to go right.
Predicted Emotion: disappointment
Mental Health Exercise: 1. Write about a time when you felt disappointment. What happened? How

In [8]:
from unsloth import FastLanguageModel
import torch

# Model loading parameters
max_seq_length = 2048  # Maximum sequence length
dtype = None  # None for auto detection; choose dtype based on your GPU
load_in_4bit = True  # Use 4-bit quantization to reduce memory usage

# Load the pre-trained model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",  # 4-bit quantized model
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

# Call for_inference on the model
model = FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [10]:
def generate_prompt(emotion):
    alpaca_prompt = f"""
    Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Provide a mental health exercise for the following feeling.

    ### Input:
    {emotion}

    ### Response:
    """
    return alpaca_prompt



input = "today my crush ingnore me so badly now i am under heavy depression"


emotion = predict_emotion(input)
# Tokenize the input prompt
prompt = generate_prompt(emotion)
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
# Generate the output with the model
outputs = model.generate(**inputs, max_new_tokens=256)

# Decode the output tokens to get the text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated response
print("Generated Response:")
print(generated_text)

Generated Response:

    Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    Provide a mental health exercise for the following feeling.

    ### Input:
    sadness

    ### Response:
     For the feeling of sadness, here's a mental health exercise:

     **Exercise: "The Weight of Sadness"**

     **Objective:** To help you process and release the emotional weight of sadness.

     **Instructions:**

     1.  Find a quiet and comfortable space where you can sit or lie down without distractions.
     2.  Close your eyes and take a few deep breaths. Feel the air move in and out of your body.
     3.  Imagine a heavy weight settling onto your shoulders or chest. This weight represents the sadness you're feeling.
     4.  As you inhale, imagine fresh air and positivity entering your body, filling any empty spaces within you.
     5.  As you exhale, imagine th

In [11]:
!pip install fastapi
!pip install nest-asyncio pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.0 MB/s eta 0:00:00


In [19]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
# Initialize Flask app
app = Flask(__name__)

# Set ngrok auth token
ngrok.set_auth_token("2ptoV8PiUCgaeMLyhAXndT6FNfd_vokEASaYWrJ7rowKGcJe")

# -------------------------------
# Define the API Endpoint
# -------------------------------

@app.route('/generate', methods=['POST'])
def generate_endpoint():
    """
    API endpoint to generate a mental health exercise based on input text.
    Expects a JSON payload with 'input_text'.
    """
    # Get the input data from the POST request
    data = request.get_json()
    if not data or 'input_text' not in data:
        return jsonify({"error": "Missing 'input_text' in request data"}), 400

    # Extract the input text
    input_text = data['input_text']

    # Step 1: Predict Emotion
    emotion = predict_emotion(input_text)
    print(f"Predicted Emotion: {emotion}")  # Optional: Log the predicted emotion

    # Step 2: Generate Mental Health Exercise based on Emotion
    exercise = generate_exercise(emotion)
    print(f"Generated Exercise: {exercise}")  # Optional: Log the generated exercise

    # Return the response as JSON
    return jsonify({"response": exercise, "emotion": emotion })

# -------------------------------
# Run the App with Ngrok
# -------------------------------

if __name__ == '__main__':
    # Open a public URL for the Flask app using Ngrok
    public_url = ngrok.connect(5000)
    print(f"Ngrok tunnel available at: {public_url}")

    # Start the Flask app
    app.run(port=5000)

Ngrok tunnel available at: NgrokTunnel: "https://d585-34-19-108-77.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Predicted Emotion: sadness


INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 16:07:02] "POST /generate HTTP/1.1" 200 -


Generated Exercise: Here's a mental health exercise to help manage feelings of sadness:

**Exercise: "Self-Care Compassion"**

**Objective:** To cultivate self-compassion and comfort when feeling sad.

**Step 1: Acknowledge Your Emotions**
Recognize and accept your feelings of sadness. It's okay to feel this way. Avoid suppressing or denying your emotions, as this can exacerbate the issue.

**Step 2: Ground Yourself**
Take a few deep breaths, focusing on the sensation of the air moving in and out of your body. This simple act can help calm your nervous system. You can also try a quick body scan: notice the sensation of your feet touching the ground, the feeling of the chair beneath you, or
Predicted Emotion: neutral


INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 16:07:22] "POST /generate HTTP/1.1" 200 -


Generated Exercise: Here's a mental health exercise to help you cope with feeling neutral:

**Exercise:** "Exploring the Middle Ground"

**Objective:** To acknowledge and work with the feeling of neutrality, allowing you to understand and possibly shift towards a more positive or balanced state.

**Step 1: Acknowledge Neutrality**
Recognize and accept your current feeling of neutrality. It's okay to feel this way. Acknowledge it without judgment.

**Step 2: Identify Triggers**
Reflect on the situations, people, or events that led you to this state of neutrality. Are you feeling disconnected or just not engaging emotionally? Understanding the triggers can help you address them directly.

**Step 3: Practice Self-Compassion**
Treat yourself with
